# Trackman Code Challenge
## Exploratory Data Analysis
We are going to extract the raw data for the challenge and assess at a high level.

In [1]:
import pandas as pd
import numpy as np
import tarfile
import json
from ast import literal_eval
import re
from pprint import pprint

From the challenge description, we know that the data consists of a **series** of configuration files in `.json` format. Each file conatains a *SELECT*, *FROM*, and *WHERE* clause. From the .json we need to programmatically determine relationships between tables.

#### Load Data File

In [2]:
tar = tarfile.open('./tables.tar.gz', "r:gz")
for member in tar.getmembers():
     f = tar.extractfile(member)
     if f is not None:
         content = f.read()

In [3]:
table_list = tar.getnames()
print(type(table_list))

<class 'list'>


#### How many files are there?

In [4]:
print(len(tar.getmembers()))

38


There are a total of 38 files that we'll be interpreting. A sample of one of the queries is below:

In [5]:
#Using literal_eval to convert binary file to dictionary
data = literal_eval(content.decode('utf8'))

In [6]:
print(type(data))
print(data)

<class 'dict'>
{'query': {'L': [{'M': {'group_by': {'S': 'a.org, s.game_id, g.game_reference, s.location_id, a.location, s.level_name, a.level, s.local_time_string, a.exception, a.access_start, a.access_end, s.game_state, s.home_team_name, a.rule_start_time, a.end_date, a.rule_end_time'}, 'where': {'S': "a.season = 2019 and s.local_time_string like '2019%' and (s.local_time_string > a.rule_start_time or left(local_time_string, 10) = left(a.rule_start_time, 10)) and (s.local_time_string < a.rule_end_time or s.local_time_string = a.rule_end_time) and (s.local_time_string > a.access_start or a.access_start is null) and (s.local_time_string < a.access_end or a.access_end is null) and a.exception = false"}, 'from': {'S': 'report.schedule_new_level s left join games.metadata g on s.game_id = g.game_id left join report.exp_access_rules_hist a on (a.level is null or a.level = s.level_name) and (a.location is null or s.location_id = a.location) and (a.team is null or s.home_team_id = a.team)'},

## Deconstructing the Configuration File
Now that we have the file in a workable format (i.e. python dictionary) we can explore how to find the relationships that we want to identify. First let's see what the *key* values are...

In [29]:
print(data.keys())

dict_keys(['schema', 'table', 'query'])


#### Schema

In [27]:
print(data['schema']['S'])

report


#### Table

In [31]:
print(data['table']['S'])

scheduled_no_exceptions_hist


#### Query
The query is the item that will require some analysis. We'll need to further deconstruct the **query** value.  

The query is made up of key-value pairs where the **key** is the SQL clause and a **value** is a data table or column.

*Note: Using the *literal_eval* is possibly not the best way to decode our .json binary files. Something to re-visit.*

In [50]:
print(type(data['query']['L'][0]))
print(data['query']['L'][0]['M'].keys(), "\n")
print("Sample Data SELECT String: ", "\n")
print(data['query']['L'][0]['M']['select']['S'], "\n")

print("Sample Data GROUP BY String: ", "\n")
print(data['query']['L'][0]['M']['group_by']['S'], "\n")

print("Sample Data WHERE String: ", "\n")
print(data['query']['L'][0]['M']['where']['S'], "\n")

print("Sample Data FROM String: ", "\n")
print(data['query']['L'][0]['M']['from']['S'], "\n")

<class 'dict'>
dict_keys(['from', 'group_by', 'where', 'select']) 

Sample Data SELECT String:  

concat(a.org, s.game_id) as orggame, a.org, s.game_id, coalesce(a.location, s.location_id) as location_id, coalesce(a.level, s.level_name) as level, s.local_time_string, g.game_reference, a.access_start, a.access_end, a.exception, s.game_state, s.home_team_name, a.rule_start_time, a.end_date, a.rule_end_time, getdate() as date_processed 

Sample Data GROUP BY String:  

a.org, s.game_id, g.game_reference, s.location_id, a.location, s.level_name, a.level, s.local_time_string, a.exception, a.access_start, a.access_end, s.game_state, s.home_team_name, a.rule_start_time, a.end_date, a.rule_end_time 

Sample Data WHERE String:  

a.season = 2019 and s.local_time_string like '2019%' and (s.local_time_string > a.rule_start_time or left(local_time_string, 10) = left(a.rule_start_time, 10)) and (s.local_time_string < a.rule_end_time or s.local_time_string = a.rule_end_time) and (s.local_time_string

We are really only concerned with the **tables** and their **relationships** in this file. We want to write a function such that it will record the table and its dependencies for each file.

In [46]:
def table_scrape(q):
    """
    Determine the table dependencies from a FROM query.
    
    Input: q (str) - snippet of SQL query
    Output: dependencies (list) - list of table dependencies
    
    utilizes re module
    """
    ## Initialize list of dependencies
    dependencies = []
    words = q.split(' ') # split query on spaces
    for word in words:
        if '.' in word:
            for t in table_list:
                if word in t and word not in dependencies:
                    dependencies.append(word)
    return (dependencies)

In [47]:
sample_from_statement = data['query']['L'][0]['M']['from']['S']
print(table_scrape(sample_from_statement))

['report.schedule_new_level', 'games.metadata', 'report.exp_access_rules_hist']


### Test Cases
A few test cases to ensure that the *table_scrape* function works properly.

In [48]:
## games.metadata.json 

print(get_config_string('tables/games.metadata.json')['query']['L'][0]['M']['from']['S'])
table_scrape(get_config_string('tables/games.metadata.json')['query']['L'][0]['M']['from']['S'])

base.games g inner join base.locations l on g.location_id = l.location_id left join rundown.location_history h on g.location_id = h.location_id AND g.date_created BETWEEN h.start_date AND h.end_date and g.application_type <> 3 AND g.game_reference NOT LIKE '%-BP-%'


['rundown.location_history']

In [49]:
## rundown.location_history_rows.json

print(get_config_string('tables/rundown.location_history_rows.json')['query']['M']['from']['S'])
table_scrape(get_config_string('tables/rundown.location_history_rows.json')['query']['M']['from']['S'])

rundown.locations


[]

In [50]:
## report.exp_access_rules_hist

print(get_config_string('tables/report.exp_access_rules_hist.json')['query']['L'][0]['M']['from']['S'])
table_scrape(get_config_string('tables/report.exp_access_rules_hist.json')['query']['L'][0]['M']['from']['S'])

 rundown.access_rules_end a full outer join broadcast.priority_lists p on a.season = p.season and (a.org = p.priority or a.org = p.org)


[]

## Populate First Level Dependencies
Now we need to build a reference dictionary for each table that contains a list of its dependencies.

In [53]:
def populate_first_level(table_list):
    """
    Passes a list of tables (i.e. configuration files) and returns a dictionary with first-level dependencies.
    
    Input: table_list (list) - list of strings
    Output: first_level_dict - dictionary of key (str) = table name and value (list) = list of dependencies.
    """
    first_level_dict = dict() # initialize dictionary
    for t in table_list:      # loop through each table
        #print(t)
        q = get_config_string(t) # load config file into memory
        #print(q['query'])
        try:
            dep = table_scrape(q['query']['L'][0]['M']['from']['S'])
        except:
            dep = table_scrape(q['query']['M']['from']['S'])
        first_level_dict[t] = dep
    return (first_level_dict)

In [54]:
#print(table_list[1:5])
first_level_dependencies = populate_first_level(table_list[1:])

In [27]:
#print(table_list)
for t in table_list:
    if 'base' in t:
        print(t)

In [61]:
words = sample_from_statement.split(' ')
for word in words:
    if '.' in word:
        i = word.find(".")
        if len(word[0:i]) > 2:
            print(word)

report.schedule_new_level
games.metadata
report.exp_access_rules_hist


In [127]:
print(get_config_string('rundown.location_history'))

{'schema': {'S': 'rundown'}, 'table': {'S': 'location_history'}, 'query': {'M': {'from': {'S': 'rundown.location_history_rows l inner join rundown.location_history_rows r on l.location_id = r.location_id and ((l.row_num = r.row_num + 1 and l.combo_id <> r.combo_id) or (l.row_num = 1 and r.row_num = 1))'}, 'select': {'S': 'l.combo_id, l.location_id, l.oem_version, l.software_version, l.radar_serial, l.radar_model, l.radar_configuration, l.radar_measurement_mode, l.start_date, coalesce(lead(l.start_date) over (partition by l.location_id order by l.location_id, l.start_date),getdate()+1) as end_date, getdate() as date_processed'}}}}


To go further down the dependency tree, we'll need to extract configuration files by member name. Let's write this out

In [68]:
config_file_name = 'tables/report.schedule_new_level.json'
temp_data = tar.extractfile(tar.getmember(config_file_name))
temp_data = temp_data.read()
temp_data = literal_eval(temp_data.decode('utf8'))

In [19]:
## writing a function to help getting data
def get_config_string(p_file_name):
    """
    Simple function to get .json string with associated configuration file name
    
    Input: p_file_name (str) - partial file name (table name)
    Output: json_string (str) - file as a json string
    """
    try:
        #file_name = 'tables/' + p_file_name + '.json'
        file_name = p_file_name
        tar = tarfile.open('./tables.tar.gz', "r:gz")
        json_string = tar.extractfile(tar.getmember(file_name))
        json_string = json_string.read()
        json_string = literal_eval(json_string.decode('utf8'))

        return (json_string)
    except:
        return (None)

In [96]:
print(get_config_string('report.schedule_new_level'))

{'schema': {'S': 'report'}, 'table': {'S': 'schedule_new_level'}, 'query': {'L': [{'M': {'from': {'S': 'lineup.schedule'}, 'where': {'S': "left(local_time_string, 4) = '2019'"}, 'select': {'S': "id, game_id, foreignid, local_time, local_time_string, time_zone, game_no, location_id, location_name, league_id, league_name, case when level_name = 'D1' then 'NCAA' when level_name = 'KoreaBaseballOrganization' then 'KBO' when level_name = 'NPM' then 'NPB Minors' else level_name end as level_name, home_team_id, home_team_name, home_team_foreignid, away_team_id, away_team_name, away_team_foreignid, game_state, getdate() as date_processed"}}}]}}


## Printing to Terminal
We'll want an output to the terminal that is human readable. Here we'll write a function to accomplish this.

In [110]:
### print current output
print(sample_output_dict)

{'dependencies': [{'dependencies': [{'dependencies': [{'dependencies': [], 'table': 'lineup.schedule'}], 'table': 'report.schedule_new_level'}], 'table': 'report.schedule_new_level'}, {'dependencies': [{'dependencies': [{'dependencies': [], 'table': 'base.games'}, {'dependencies': [], 'table': 'base.locations'}, {'dependencies': [], 'table': 'rundown.location_history'}], 'table': 'games.metadata'}], 'table': 'games.metadata'}, {'dependencies': [{'dependencies': [{'dependencies': [], 'table': 'rundown.access_rules_end'}, {'dependencies': [], 'table': 'broadcast.priority_lists'}], 'table': 'report.exp_access_rules_hist'}], 'table': 'report.exp_access_rules_hist'}], 'table': 'report.scheduled_no_exceptions_hist'}


In [113]:
print(sample_output_dict['table'], '\n', '\t|-', sample_output_dict['dependencies'][0]['table'])

report.scheduled_no_exceptions_hist 
 	|- report.schedule_new_level


In [115]:
print(sample_output_dict['table'], '\n')
for i in range(len(sample_output_dict['dependencies'])):
    print('\t|\n\t|+', sample_output_dict['dependencies'][i]['table'])

report.scheduled_no_exceptions_hist 

	|
	|+ report.schedule_new_level
	|
	|+ games.metadata
	|
	|+ report.exp_access_rules_hist


In [74]:
def print_dep_table(name, n=0):
    """
    name (str) - table name
    n (int) - number of levels
    """
    if n == 0:
        print(name, '\n', file=open("output.txt", "a"))
    else:
        print('\t'*n, '|\n', '\t'*n, '|+', name, file=open("output.txt", "a"))

In [64]:
names = list(first_level_dependencies.keys())
print_dep_table(names[0][7:-5], 0)

report.delivered_combined_hist 



In [67]:
def pretty_print_dep_table(t, dep_table, n=0):
    print_dep_table(t[7:-5], n)
    if len(first_level_dependencies[t]) > 0:
        n += 1
        for dep in dep_table[t]:
            pretty_print_dep_table('tables/' + dep + '.json', n)

In [76]:
output_file.close()